# Projeto 1 - MS211 - Calculo Numérico

### Aluno:
Luiz Eduardo Cartolano - RA: 183012

### Professor:
Maicon R. Correa

### Github Link:
[Project Repository](https://github.com/luizcartolano2/ms211-numerical-calculus)



# Primeira Questão
Projeto 1 - Método de Newton Discreto, do livro Cálculo Numérico, Ruggiero-Lopes, 2a Ed, (p. 206).

In [2]:
# import das librarys usadas
import numpy as np

### Função de Rosenbrock - F(x)

A primeira questão envolve resolver a função de Rosenbrock usando duas versões do método de Newton, o tradicional e o modificado. A função de Rosenbrock é dada pelo seguinte sistema:
\begin{align}
    - 10 \cdot (x_1)^2 + 10 \cdot x_2 = 0 \\
    1 - x_1 = 0
\end{align}

A função pode ser definida em python do seguinte modo:

In [3]:
def F(x):
    return np.array(
        [[-10 * x[0][0] ** 2 + 10 * x[0][1],
        1 - x[0][0]]]
    ).reshape(2,)

### Jacobiana - J(x)
A fim de não ser preciso calcular a Jacobiana, que é definida como:

\begin{equation*}
    \mathbf{J} =  \begin{vmatrix}
        \frac{\partial f_1(x)}{\partial x_1} &  \frac{\partial f_1(x)}{\partial x_2} \\
        \frac{\partial f_2(x)}{\partial x_1} &  \frac{\partial f_2(x)}{\partial x_2} 
    \end{vmatrix}
\end{equation*}

O método de Newton Discreto realiza uma aproximação, que calcula a jacobiana a partir de um método discreto, para isso, é preciso definir primeiro:

\begin{equation*}
    e_j = (0,0,...1,0,0,...0)^T
\end{equation*}
onde a posição $j$ tem o valor 1. E a coluna $j$ da jacobiana será dada por:

\begin{equation*}
    \frac{\mathbf{F}(x + he_j) - \mathbf{F}(x)}{h}
\end{equation*}

A duas funções seguintes foram capaz de implementar a jacobiana utilizando Python.

In [4]:
def e_(j, size):
    e_t = np.zeros((size,1))
    e_t[j] = 1
    
    return e_t

In [5]:
def J(x, h):
    f_1 = ((F(x + e_(0,2).T * h) - F(x)).T)/h
    f_2 = ((F(x + e_(1,2).T * h) - F(x)).T)/h
    
    return np.column_stack((f_1, f_2))

### Método de Newton

O método de Newton consiste em, dado o ponto $x^{(k)}$, a matriz $J(x^{(k)})$ é obtida avaliando-se $J(x)$ em $x^{(k)}$ e, em seguida, o passo de Newton, $s^{(k)}$, é obtido a partir da resolução do sistema linear, $J(x^{(k)})s = -F(x^{(k)})$. Portanto, uma iteração de Newton requer que:

1. a avaliação da matriz Jacobiana em $x^{(k)}$
2. a resolução do sistema linear $J(x^{(k)})s = -F(x^{(k)})$

O algoritmo do método de Newton é dado por:

Dados $x_0$, $\epsilon_1 > 0$ e $\epsilon_2 > 0$:

1. calcule $F(x^{(k)})$ e $J(x^{(k)})$
2. se $|F(x^{(k)})| < \epsilon_1$, faça $\bar{x} = x^{(k)}$ e pare
3. obtenha $s^{(k)}$, a solução do sistema linear $J(x^{(k)})s = -F(x^{(k)})$
4. faça: $x^{(k+1)} = x^{(k)} + s^{(k)}$


Abaixo segue a implementação do método em Python.

In [19]:
def newton(F, J, x0, h, e1=1e-4, e2=1e-4):
    counter = 0
    x = x0
    if abs(np.max(F(x))) < e1:
        return x0
    s = np.linalg.solve(J(x,h),-F(x))
    xk = x + s
    
    while abs(np.max(F(x))) >= e1:
        counter = counter + 1
        x = xk
        s = np.linalg.solve(J(x,h),-F(x))
        xk = x + s
    
    return xk, counter

### Método de Newton Modificado

A alteração feita para o método de Newton consiste em tomar a cada iteração $k$ a matriz $J(x^{(0)})$, em vez de $J(x^{(k)})$: a partir de uma aproximação inicial $x^{(0)}$, a sequência $\{x^{(k)}\}$ é gerada através de $x^{(k+1)} = x^{(k)} + s^{(k)}$, onde $s^{(k)}$ é a solução do sistema linear: 

\begin{equation*}
    J(x^{(0)})s = -F(x^{(k)})
\end{equation*}

Desta forma, a matriz Jacobiana é avaliada apenas uma vez e, para todo $k$, o sistema linear a ser resolvido terá a mesma matriz Jacobiana. A implementação em Python pode ser feita, com poucas alterações para a função anterior, como mostrado abaixo.

In [32]:
def newton_modificado(F, J, x0, h, e1=1e-10, e2=1e-4):
    counter = 0
    x = x0
    if abs(np.max(F(x))) < e1:
        return x0
    
    J_ = J(x,h)
    s = np.linalg.solve(J_,-F(x))
    xk = x + s
    
    while abs(np.max(F(x))) >= e1: # and abs(np.max(xk-x)) >= e2:
        counter = counter + 1
        x = xk
        s = np.linalg.solve(J_,-F(x))
        xk = x + s
    
    return xk, counter

A seguir a solução do sistema para $h = 10^{-2}$ usando o método de Newton tradicional.

In [47]:
x, iter_ = newton(F=F, J=J, x0=np.array([[-1.2, 1]]), h=1e-2)
print("A solução para equação é dada por: ({},{}), e foram precisas {} iterações.".format(x[0][0],x[0][1],iter_))

A solução para equação é dada por: (1.0,0.9999999999999325), e foram precisas 1 iterações.


A seguir a solução do sistema para $h = 10^{-5}$ usando o método de Newton tradicional.

In [49]:
x, iter_ = newton(F=F, J=J, x0=np.array([[-1.2, 1]]), h=1e-5)
print("A solução para equação é dada por: ({},{}), e foram precisas {} iterações.".format(x[0][0],x[0][1],iter_))

A solução para equação é dada por: (0.9999999999999998,1.0000000001832312), e foram precisas 1 iterações.


A seguir a solução do sistema para $h = 10^{-2}$ usando o método de Newton modificado.

In [48]:
x, iter_ = newton_modificado(F=F, J=J, x0=np.array([[-1.2, 1]]), h=1e-2)
print("A solução para equação é dada por: ({},{}), e foram precisas {} iterações.".format(x[0][0],x[0][1],iter_))

A solução para equação é dada por: (0.9999999999999997,0.9999999999997948), e foram precisas 1 iterações.


A seguir a solução do sistema para $h = 10^{-5}$ usando o método de Newton modificado.

In [50]:
x, iter_ = newton_modificado(F=F, J=J, x0=np.array([[-1.2, 1]]), h=1e-5)
print("A solução para equação é dada por: ({},{}), e foram precisas {} iterações.".format(x[0][0],x[0][1],iter_))

A solução para equação é dada por: (1.0,0.9999999999999984), e foram precisas 2 iterações.


# Segunda Questão